In [26]:
import numpy as np
from scipy.ndimage import distance_transform_edt

def add_surface_noise(volume : np.ndarray, amount : float, seed=None):

    # Setting the seed if provided
    if seed is not None:
        np.random.seed(seed)

    # Calculate the distance transform of the volume
    distance = distance_transform_edt(1 - volume)

    # Extracting the surface where the distance to the volume is one voxel away
    surface = np.where(distance==1, 1, 0)

    # Generate random noise 
    noise = np.random.rand(*volume.shape)

    # Binarizing a given proportion of noisy voxels
    noise = np.where(noise >= amount, 0, 1)

    # Masking to the surface
    noise = noise * surface

    # Adding the noise to the original volume
    noisy_volume = volume + noise

    # Returing the ndarray as 8-bit unsigned integer type
    return noisy_volume.astype(np.uint8)

# Example usage:
if __name__ == "__main__":
    # Assuming you have your binarized 3D numpy array volume
    seed = 42
    volume = np.zeros((10, 10, 10), dtype=np.uint8)
    volume[3:7, 3:7, 3:7] = 1  # Example: Creating a small solid volume in the center
    amount = 0.1  # Example: Noise amount (between 0 and 1)

    noisy_volume = add_surface_noise(volume, amount, seed)
    noisy_volume2 = add_surface_noise(volume, amount, seed)
    print(np.any(noisy_volume2-noisy_volume))



True


In [49]:
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from pathlib import Path

with np.load('/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/processed/stn_space_3sigma/merged/flipped/VTAs/250um.npz') as f:
    X = f['arr_0']

noisy_volumefactor = 1.2
n_noisy = int(np.floor(len(X)*noisy_volumefactor))

random_indices = np.random.randint(0, len(X), size=n_noisy)

noisy_volumes = []

random_selections = [X[i] for i in random_indices]

df_dict = {
    'idx' : list(),
    'original_VTA_voxels' : list(),
    'total_voxels' : list(),
    'added_voxels' : list(),
    'source_nifti' : list()

}

table = '/home/brainstimmaps/RESEARCH/20xx_Projects/2009_DeepMaps01/deepmaps/data/raw/tables/stn_space/merged/flipped/table_zeroed.csv'

try:
    df = pd.read_csv(table)
except:
    df = None
    print(f'Cannot read table : {table}')

for i,VTA in enumerate(tqdm(random_selections)):
    amount = np.random.uniform(0.1, 0.5)
    noisy_volume = add_surface_noise(VTA, amount, seed)
    noisy_volumes.append(noisy_volume)

    vta_idx = random_indices[i]
    try:
        source_nifti = df.at[vta_idx, 'massive_filename']
    except:
        print(f'Could not retreive VTA {i} massive filename in table : {table}')
        source_nifti = ''

    df_dict['idx'].append(vta_idx)
    df_dict['original_VTA_voxels'].append(np.sum(VTA))
    df_dict['added_voxels'].append(np.sum(noisy_volume-VTA))
    df_dict['total_voxels'].append(np.sum(noisy_volume))
    df_dict['source_nifti'].append(source_nifti)

noisy_volumes = np.array(noisy_volumes)

added_noises = noisy_volumes - random_selections

output_path = f'{noisy_volumefactor}factor_stn_space_3sigma_merged_flipped_250um/noisy_volumes.npz'
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
np.savez_compressed(f'{noisy_volumefactor}factor_stn_space_3sigma_merged_flipped_250um/noisy_volumes.npz', noisy_volumes)
np.savez_compressed(f'{noisy_volumefactor}factor_stn_space_3sigma_merged_flipped_250um/added_noises.npz', added_noises)

now = datetime.now()
    
dt_string = now.strftime("%Y_%m_%d_%H_%M_%S")

csv_file = f'{noisy_volumefactor}factor_stn_space_3sigma_merged_flipped_250um/log_{dt_string}.csv'

df_log = pd.DataFrame(df_dict)
df_log.to_csv(csv_file, index=False)

# approx 2minutes for 8395 VTAs and 1.2 noisy factor (10k volumes)

100%|██████████| 10314/10314 [01:59<00:00, 86.01it/s]


In [46]:
# No noisy volume is smaller than 0
np.array([np.sum(noisy_volume) < 0 for noisy_volume in noisy_volumes]).any()

False

In [47]:
# No added noise volume is smaller than zero
np.array([np.sum(noise) < 0 for noise in added_noises]).any()

False